In [1]:
# Importing libraries

import psycopg2 as ps
import pandas as pd
import warnings
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{0:,.0f}'.format

def wms_valuation():
    conn = ps.connect(user = 'doadmin', password = 'xpmt05ij9uf9rknn', host = 'tjori-bi-do-user-6486966-0.db.ondigitalocean.com', port = '25060', database = 'defaultdb')
    sql = '''SELECT 
                b.category AS category
               ,b.sku AS product_sku
               ,b.size AS product_size
               ,COUNT(a.in_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') AS inward_stock
               ,COUNT(a.out_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') AS outward_stock
               ,COUNT(a.in_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') - COUNT(a.out_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') AS quantity
               ,coalesce(sp.special_price,0) AS price
               ,((COUNT(a.in_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') - COUNT(a.out_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata')) * (coalesce(sp.special_price,0))) AS stock_valuation
    
            FROM 
               inventory_skubin a
               LEFT JOIN inventory_skuean b ON a.sku_id = b.id
               LEFT JOIN store_product AS sp ON b.sku = sp.sku

           GROUP BY
               b.category
              ,b.sku
              ,b.size
              ,sp.special_price

          ORDER BY
              quantity DESC

;		
'''

    df = pd.read_sql_query(sql,conn)
    return df
#WMS Stock Valuation

dfw = wms_valuation()

#WMS Quantity

dfw1w1 = dfw[dfw['category'] == 'apparel'].groupby('category')['quantity'].sum()
dfw11w11 = dfw1w1.to_string(index = None, header = None)

dfw2w2 = dfw[dfw['category'] == 'bags'].groupby('category')['quantity'].sum()
dfw22w22 = dfw2w2.to_string(index = None, header = None)

dfw3w3 = dfw[dfw['category'] == 'footwear'].groupby('category')['quantity'].sum()
dfw33w33 = dfw3w3.to_string(index = None, header = None)

dfw4w4 = dfw[dfw['category'] == 'home'].groupby('category')['quantity'].sum()
dfw44w44 = dfw4w4.to_string(index = None, header = None)

dfw5w5 = dfw[dfw['category'] == 'jewelry'].groupby('category')['quantity'].sum()
dfw55w55 = dfw5w5.to_string(index = None, header = None)

dfw6w6 = dfw[dfw['category'] == 'living'].groupby('category')['quantity'].sum()
dfw66w66 = dfw6w6.to_string(index = None, header = None)

dfw7w7 = dfw[dfw['category'] == 'accessories'].groupby('category')['quantity'].sum()
dfw77w77 = dfw7w7.to_string(index = None, header = None)

dfw8w8 = dfw[dfw['category'] == 'motherchild'].groupby('category')['quantity'].sum()
dfw88w88 = dfw8w8.to_string(index = None, header = None)

#WMS Stock_valuation

dfm1m1 = dfw[dfw['category'] == 'apparel'].groupby('category')['stock_valuation'].sum()
dfm11m11 = dfm1m1.to_string(index = None, header = None)

dfm2m2 = dfw[dfw['category'] == 'bags'].groupby('category')['stock_valuation'].sum()
dfm22m22 = dfm2m2.to_string(index = None, header = None)

dfm3m3 = dfw[dfw['category'] == 'footwear'].groupby('category')['stock_valuation'].sum()
dfm33m33 = dfm3m3.to_string(index = None, header = None)

dfm4m4 = dfw[dfw['category'] == 'home'].groupby('category')['stock_valuation'].sum()
dfm44m44 = dfm4m4.to_string(index = None, header = None)

dfm5m5 = dfw[dfw['category'] == 'jewelry'].groupby('category')['stock_valuation'].sum()
dfm55m55 = dfm5m5.to_string(index = None, header = None)

dfm6m6 = dfw[dfw['category'] == 'living'].groupby('category')['stock_valuation'].sum()
dfm66m66 = dfm6m6.to_string(index = None, header = None)

dfm7m7 = dfw[dfw['category'] == 'accessories'].groupby('category')['stock_valuation'].sum()
dfm77m77 = dfm7m7.to_string(index = None, header = None)

dfm8m8 = dfw[dfw['category'] == 'motherchild'].groupby('category')['stock_valuation'].sum()
dfm88m88 = dfm8m8.to_string(index = None, header = None)


#WMS Sum Total

#Sum_Total

dfs1s1 = format(sum(dfw['quantity']),",d")

dfs2s2 = format(int(sum(dfw['stock_valuation'])),",d")

# Initiating the STMP

sender = "nimit@tjori.com"
recievers = ["nimit@tjori.com","ankit@tjori.com","mansi@tjori.com","poonam@tjori.com"]

# Create message container - 
msg = MIMEMultipart('alternative')
msg['Subject'] = "Daily WMS Stock Report"
msg['From'] = sender
msg['To'] = ",".join(recievers)

# Create the body of the message 

html = """\
<html>
  <head>
  <style> 
    table, th, td { 
    border: 2px solid black; 
    border-collapse: collapse; 
    } 
    th, td { 
        padding: 4px;
        background-color: #dff5f4;
        text-align: center;
    }
  </style> 
  </head>
  <body>
  <p><b>WMS Stock Valuation</b></p>
  <table style="width:100%" border = 0; padding = 0;>
    <col width = "200">
    <col width = "200">
    <col width = "200">
  <tr>
    <th style="background-color:#000; color:#fff; padding: 10px;"> Category Name </th>
    <th style="background-color:#000; color:#fff; padding: 10px;">Stock Quantity</th>
    <th style="background-color:#000; color:#fff; padding: 10px;">Stock Valuation</th>
  </tr>
  <tr>
  <td><b>Apparel</b></td>
  <td>""" + str(dfw11w11) + """</td>
  <td>""" + str(dfm11m11) + """</td>
  </tr>
  <tr>
  <td><b>Bags</b></td>
  <td>""" + str(dfw22w22) + """</td>
  <td>""" + str(dfm22m22) + """</td>
  </tr>
  <tr>
  <td><b>Footwear</b></td>
  <td>""" + str(dfw33w33) + """</td>
  <td>""" + str(dfm33m33) + """</td>
  </tr>
  <tr>
  <td><b>Home & Decor</b></td>
  <td>""" + str(dfw44w44) + """</td>
  <td>""" + str(dfm44m44) + """</td>
  </tr>
  <tr>
  <td><b>Jewelry</b></td>
  <td>""" + str(dfw55w55) + """</td>
  <td>""" + str(dfm55m55) + """</td>
  </tr>
  <tr>
  <td><b>Wellness</b></td>
  <td>""" + str(dfw66w66) + """</td>
  <td>""" + str(dfm66m66) + """</td>
  </tr>
  <tr>
  <td><b>Accessories</b></td>
  <td>""" + str(dfw77w77) + """</td>
  <td>""" + str(dfm77m77) + """</td>
  </tr>
  <tr>
  <td><b>Mother & Child</b></td>
  <td>""" + str(dfw88w88) + """</td>
  <td>""" + str(dfm88m88) + """</td>
  </tr>
  <tr>
  <td style="background-color:#000; color:#fff; padding: 10px;"><b>Sum total</b></td>
  <td style="background-color:#000; color:#fff; padding: 10px;"><b> """+str(dfs1s1)+"""</b></td>
  <td style="background-color:#000; color:#fff; padding: 10px;"><b> """+str(dfs2s2)+"""</b></td>
  </tr>
  </table>
  </body>
</html>
"""

# Record the MIME type - 
part1 = MIMEText(html, 'html')

msg.attach(part1)

# Send the message via local SMTP server - 
mail = smtplib.SMTP('smtp.gmail.com', 587)

mail.ehlo()

mail.starttls()

mail.login('nimit@tjori.com', 's/987456321/G')
mail.sendmail(sender, recievers, msg.as_string())
mail.quit()






(221, b'2.0.0 closing connection k84sm21720220pfd.157 - gsmtp')